In [1]:
#Install OpenAI
# !pip install openai

In [2]:
import src 
import pandas as pd
import os

In [3]:
# from importlib import reload
# reload(src)

In [4]:
DATA_PATH = "data"
SAMPLE_DATASET_SIZE = 1000
ORIGINAL_PICKLE_FILENAME = "ct_df_posts_single_preds.pkl"


def get_sampled_dataset(
    original_pickle_filename,
    sample_size,
    random_state=42,
    keep_columns=["caption", "is_ad"],
):
    dataset_filename = f"{DATA_PATH}/sample_{sample_size}_{random_state}.csv"
    if os.path.exists(dataset_filename):
        print("loading sample dataset...")
        sample_dataset = pd.read_csv(dataset_filename)
    else:
        print("creating sample dataset...")
        full_df_filename = f"{DATA_PATH}/{original_pickle_filename}"
        full_df = pd.read_pickle(full_df_filename)

        sponsored = full_df[full_df["is_ad"]].sample(
            sample_size, random_state=random_state
        )
        unsponsored = full_df[~full_df["is_ad"]].sample(
            sample_size, random_state=random_state
        )
        sample_dataset = pd.concat([sponsored, unsponsored])[keep_columns]
        sample_dataset.to_csv(dataset_filename, index=False)
    return sample_dataset


def format_examples(examples: list) -> str:
    return "".join(
        [
            f"{index + 1}. <POST> {example}</POST>\n"
            for index, example in enumerate(examples)
        ]
    )


def get_caption_examples(number_examples, is_ad):
    sample_dataset = get_sampled_dataset(
        original_pickle_filename=ORIGINAL_PICKLE_FILENAME,
        sample_size=int(SAMPLE_DATASET_SIZE / 2),
    )
    examples = (
        sample_dataset[sample_dataset["is_ad"]]["caption"]
        .sample(number_examples)
        .tolist()
    )
    examples = [example.replace("\n", "") for example in examples]
    return format_examples(examples)

# Example 1

## create experiment

In [9]:
experiment_identifier = "haiku_experiement"
prompt_name = "write_me_a"
parameters = {"text_type": "haiku", "max_length": 4}

experiment_results, setup_experiment = src.create_experiment(
    experiment_identifier, prompt_name, parameters
)
display(experiment_results)
display(setup_experiment)

Running experiment haiku_experiement with prompt write_me_a and parameters {'text_type': 'haiku', 'max_length': 4}
	 > Sending messages to OpenAI API...
	 > Received response from OpenAI API
	 > processing response...
	 > Storing results...
Experiment complete!


,0
0,"Moon shines, night is calm"
1,"Whispers float, love's sweet balm"
2,"Hearts entwined, soul's psalm"


{'experiment_identifier': 'haiku_experiement',
 'prompt_name': 'write_me_a',
 'df_results': 'results/haiku_experiement/df_haiku_experiement.csv',
 'parameters': {'text_type': 'haiku', 'max_length': 4}}

## read experiment

In [10]:
experiment_identifier = "haiku_experiement"

experiment_results, setup_experiment = src.load_results(experiment_identifier)
display(experiment_results)
display(setup_experiment)

,0
0,"Moon shines, night is calm"
1,"Whispers float, love's sweet balm"
2,"Hearts entwined, soul's psalm"


{'experiment_identifier': 'haiku_experiement',
 'prompt_name': 'write_me_a',
 'df_results': 'results/haiku_experiement/df_haiku_experiement.csv',
 'parameters': {'text_type': 'haiku', 'max_length': 4}}

# example 2

### creating experiment

In [ ]:
# required values to run experiment
experiment_identifier = "unsponsored_random_examples_exp_1"
prompt_name = "unsponsored_random_examples"


# parameters to be passed to the prompt
number_of_examples_used = 3
number_of_captions_desiered = 5
examples = get_caption_examples(number_examples=number_of_examples_used, is_ad=False)

# set parameters variable
parameters = {
    "number_of_captions_given": number_of_examples_used,
    "number_of_captions_desiered": number_of_captions_desiered,
    "examples": examples,
}

# run experiment
print("Reading prompt template...")
prompt_json = src.read_prompt_json(prompt_name)

print("Calling OpenAI API...")
prompt_messages = src.create_openai_messages(prompt_json["messages"], parameters)
_, response_content, _ = src.get_completion_from_messages(prompt_messages)

print("Processing response...")
experiment_results = src.process_response_content(response_content)

setup_experiment = src.create_setup_dict(experiment_identifier, prompt_name, parameters)

print("Storing results...")
src.save_results(experiment_identifier, experiment_results, setup_experiment)

print("Experiment complete!")

### reading experiment results

In [6]:
experiment_identifier = "unsponsored_random_examples_exp_1"

experiment_results, setup_experiment = src.load_results(experiment_identifier)
display(experiment_results)
display(setup_experiment)
display(setup_experiment["parameters"]["examples"].split("\n"))

,0
0,1. <POST> Capturing the beauty of nature on th...
1,2. <POST> Celebrating the little things in lif...
2,3. <POST> Embracing my inner foodie at this hi...
3,4. <POST> Getting my creative juices flowing w...
4,5. <POST> Reflecting on the power of self-care...


{'experiment_identifier': 'unsponsored_random_examples_exp_1',
 'prompt_name': 'unsponsored_random_examples',
 'experiment_results_filename': 'results/unsponsored_random_examples_exp_1/df_unsponsored_random_examples_exp_1.csv',
 'parameters': {'number_of_captions_given': 3,
  'number_of_captions_desiered': 5,
  'examples': '1. <POST> #ad @listerine Ready! Tabs helps you keep all your favorite things close to you during the holidays. Just chew, swish & swallow and you’re ready to go for anything during the holiday season. #ReadySwishGo</POST>\n2. <POST> Dia de sol, piscina e praia 🐬 Me transformei numa sereia real com esse biquíni 🧜🏻\u200d♀️👙 @aguadocepraia ! Apaixonadaaaaa nesse modelo e na estampa!! Gostaram seus lindos? 🌈 #SereiasAguaDoce</POST>\n3. <POST> When you can’t smell shit and your husband is so excited about his new toy 😂 Luke kan echt niet ophouden over zijn fles van @airup… Door de geurpod, lijkt het alsof je limonade drinkt terwijl het gewoon puur water is. Aangezien ik 

['1. <POST> #ad @listerine Ready! Tabs helps you keep all your favorite things close to you during the holidays. Just chew, swish & swallow and you’re ready to go for anything during the holiday season. #ReadySwishGo</POST>',
 '2. <POST> Dia de sol, piscina e praia 🐬 Me transformei numa sereia real com esse biquíni 🧜🏻\u200d♀️👙 @aguadocepraia ! Apaixonadaaaaa nesse modelo e na estampa!! Gostaram seus lindos? 🌈 #SereiasAguaDoce</POST>',
 '3. <POST> When you can’t smell shit and your husband is so excited about his new toy 😂 Luke kan echt niet ophouden over zijn fles van @airup… Door de geurpod, lijkt het alsof je limonade drinkt terwijl het gewoon puur water is. Aangezien ik er zelf niet van kan genieten i.v.m. reuk, deel ik het graag met jullie! Met de code JIPP krijg je 15% korting op je starterset #partner (Code is 5 dagen geldig)</POST>',
 '']

# Example 3

### creating experiment

In [7]:
# requiered values to run experiment
experiment_identifier = "sponsored_random_examples_exp_1"
prompt_name = "sponsored_random_examples"


# parameters to be passed to the prompt
number_of_examples_used = 3
number_of_captions_desiered = 5
examples = get_caption_examples(number_examples=number_of_examples_used, is_ad=True)

# set parameters variable
parameters = {
    "number_of_captions_given": number_of_examples_used,
    "number_of_captions_desiered": number_of_captions_desiered,
    "examples": examples,
}

# run experiment
print("Reading prompt template...")
prompt_json = src.read_prompt_json(prompt_name)

print("Calling OpenAI API...")
prompt_messages = src.create_openai_messages(prompt_json["messages"], parameters)
_, response_content, _ = src.get_completion_from_messages(prompt_messages)

print("Processing response...")
experiment_results = src.process_response_content(response_content)

setup_experiment = src.create_setup_dict(experiment_identifier, prompt_name, parameters)

print("Storing results...")
src.save_results(experiment_identifier, experiment_results, setup_experiment)

print("Experiment complete!")

loading sample dataset...
Reading prompt template...
Calling OpenAI API...
Processing response...
Storing results...
Experiment complete!


### reading experiment results

In [8]:
experiment_identifier = "sponsored_random_examples_exp_1"

experiment_results, setup_experiment = src.load_results(experiment_identifier)
display(experiment_results)
display(setup_experiment)
display(setup_experiment["parameters"]["examples"].split("\n"))

,0
0,1. <POST> Sippin' on my favorite @fitfuelbev p...
1,2. <POST> Can't get enough of the stunning vie...
2,3. <POST> Sunday self-care with @blissfulbeaut...
3,4. <POST> Obsessed with the new collection fro...
4,5. <POST> Indulging in a little self-love with...


{'experiment_identifier': 'sponsored_random_examples_exp_1',
 'prompt_name': 'sponsored_random_examples',
 'experiment_results_filename': 'results/sponsored_random_examples_exp_1/df_sponsored_random_examples_exp_1.csv',
 'parameters': {'number_of_captions_given': 3,
  'number_of_captions_desiered': 5,
  'examples': "1. <POST> Glowing Skin is ALWAYS in & Banana Bright Primer leaves my skin looking luminous and young all day long 🍌 #GIVEAWAY ✨ One lucky winner AND their bestie will win an entire box of my favorite #olehenriksen ksen products! Here's how to enter:1️⃣ LIKE this post 2️⃣ Follow @olehenriksen and @jadeywadey180 3️⃣ Tag a friend to let them know about this giveaway ⭐️ Each tagged friend counts as an additional entry! You have 48 hours to win... Good luck! #BrightAway @sephora #glowcamp #skincare</POST>\n2. <POST> The @summerfridays Jet Lag Mask has been a favourite of mine for a few years now. Keeps me glowy and hydrated 🥰#summerfridayspartner</POST>\n3. <POST> Trying with my

["1. <POST> Glowing Skin is ALWAYS in & Banana Bright Primer leaves my skin looking luminous and young all day long 🍌 #GIVEAWAY ✨ One lucky winner AND their bestie will win an entire box of my favorite #olehenriksen ksen products! Here's how to enter:1️⃣ LIKE this post 2️⃣ Follow @olehenriksen and @jadeywadey180 3️⃣ Tag a friend to let them know about this giveaway ⭐️ Each tagged friend counts as an additional entry! You have 48 hours to win... Good luck! #BrightAway @sephora #glowcamp #skincare</POST>",
 '2. <POST> The @summerfridays Jet Lag Mask has been a favourite of mine for a few years now. Keeps me glowy and hydrated 🥰#summerfridayspartner</POST>',
 '3. <POST> Trying with my new @remediplus blood orange cbd topical 🌸 I’m always accidentally injuring myself or sore from the gym. This all-natural topical is perfect for soothing aches and sores. (And it’s on sale atwww.remedishop.com ;) ) #remedicbd</POST>',
 '']